In [17]:
import numpy as np
import random
import matplotlib.pyplot as plt
import copy
import gymnasium as gym
from gymnasium import spaces
from pettingzoo import ParallelEnv

num_agents = 10                   
num_rounds = 200                    
b = [2,5,10]             # Benefit (focus on b/c=5 as in paper)
c = 1                    # Cost of cooperation
chi = 0.01              # Reputation assignment error

# Learning parameters
learning_rate = 0.01
gamma = 0.99
epsilon = 0.1
num_episodes = 1000

num_seeds = 20           # Number of random seeds for each experiment

In [18]:
class MatrixGame(ParallelEnv):
    metadata = {'render_modes': ['human']}

    def __init__(self, reward_matrix, agents):
        self.agents = agents 
        self.possible_agents = self.agents[:]
        self.reward_matrix = reward_matrix
        self.norm = [0, 0, 0, 0]

    def reset(self, seed=None, options=None):
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0.0 for agent in self.agents}
        self.states = {agent: np.random.choice([0,1]) for agent in self.agents}

        return self.states
    
    def get_action_rules(self, action_rule):
        # Convert rule_id to 4-bit binary
        bits = [(action_rule >> i) & 1 for i in range(4)]  # bits[0]=LSB, bits[3]=MSB
        return bits

    def determine_action(self, agent1, agent2, action_rules):
        state1 = self.states[agent1]
        state2 = self.states[agent2]

        action_rule1 = self.get_action_rules(action_rules[agent1])
        action_rule2 = self.get_action_rules(action_rules[agent2])

        action1 = self.action_func(action_rule1, state1, state2)
        action2 = self.action_func(action_rule2, state2, state1)

        return action1, action2
            
    
    def action_func(self, action_rule, focal_state, opponent_state):
        if focal_state == 0 and focal_state == 0:
            return action_rule[3]  # Bit 3
        elif focal_state == 0 and opponent_state == 1:
            return action_rule[2]  # Bit 2
        elif focal_state == 1 and opponent_state == 0:
            return action_rule[1]  # Bit 1
        else:  # (1,1)
            return action_rule[0]  # Bit 0

    def determine_state(self, focal_action, opponent_state):
        if focal_action == 0 and opponent_state == 0:
            return self.norm[3]  # Bit 3
        elif focal_action == 0 and opponent_state == 1:
            return self.norm[2]  # Bit 2
        elif focal_action == 1 and opponent_state == 0:
            return self.norm[1]  # Bit 1
        else:  # (1,1)
            return self.norm[0]  # Bit 0

    def step(self, action_rules):
        pairings = []
        players = self.agents.copy()
        for _ in range(num_agents//2):
            index = random.randrange(len(players))
            elem1 = players.pop(index)

            index = random.randrange(len(players))
            elem2 = players.pop(index)

            pairings.append((elem1, elem2))
            

        for pair in pairings:
            action1, action2 = self.determine_action(pair[0], pair[1], action_rules)

            reward1 = self.reward_matrix[action1][action2]
            reward2 = self.reward_matrix[action2][action1]

            self.rewards[pair[0]] = reward1
            self.rewards[pair[1]] = reward2

            state1 = self.determine_state(action1, self.states[pair[1]])
            state2 = self.determine_state(action2, self.states[pair[0]])

            if (random.random() < chi):
                state1 = 1-state1
            if (random.random() < chi):
                state2 = 1-state2

            self.states[pair[0]] = state1
            self.states[pair[1]] = state2

        return self.states, self.rewards

In [19]:
class Qlearner:
    """A Q-learning agent"""

    def __init__(
        self,
        seeded=False,
        action_size=16,
        state_size=2,
        learning_rate=0.01,
        gamma=0.99,
        epsilon=0.1,
    ):
        self.action_size = action_size
        self.state_size = state_size

        self.seeded = seeded
        # initialize the Q-table: (State x Agent Action)
        self.qtable = np.zeros((self.state_size, self.action_size))

        self.learning_rate = learning_rate
        self.gamma = gamma  # discount factor
        self.epsilon = epsilon # exploration

        # tracking rewards/progress:
        self.rewards_this_episode = []  # during an episode, save every time step's reward
        self.episode_total_rewards = []  # each episode, sum the rewards, possibly with a discount factor
        self.average_episode_total_rewards = []  # the average (discounted) episode reward to indicate progress

        self.state_history = []
        self.action_history = []

    def reset_agent(self):
        self.qtable = np.zeros((self.state_size, self.action_size))

    def select_greedy(self, state):
        # np.argmax(self.qtable[state]) will select first entry if two or more Q-values are equal, but we want true randomness:
        return np.random.choice(np.flatnonzero(np.isclose(self.qtable[state], self.qtable[state].max())))

    def select_action(self, state):
        if self.seeded:
            return 5 
        if np.random.rand() < self.epsilon:
            action = random.randrange(self.action_size)
        else:
            action = self.select_greedy(state)
        self.state_history.append(state)
        self.action_history.append(action)
        return action

    def update(self, state, action, new_state, reward, done, update_epsilon=True):
        lr = self.learning_rate
        self.qtable[state, action] += lr * (reward + (not done) * self.gamma * np.max(self.qtable[new_state]) - self.qtable[state, action])

        self.rewards_this_episode.append(reward)

        if done:
            # track total reward:
            episode_reward = self._calculate_episode_reward(self.rewards_this_episode, discount=False)
            self.episode_total_rewards.append(episode_reward)

            k = len(self.average_episode_total_rewards) + 1  # amount of episodes that have passed
            self._calculate_average_episode_reward(k, episode_reward)
            
            # reset the rewards for the next episode:
            self.rewards_this_episode = []

    def _calculate_episode_reward(self, rewards_this_episode, discount=False):
        if discount:
            return sum([self.gamma**i * reward for i, reward in enumerate(rewards_this_episode)])
        return sum(rewards_this_episode)

    def _calculate_average_episode_reward(self, k, episode_reward):
        if k > 1:  # running average is more efficient:
            average_episode_reward = (1 - 1 / k) * self.average_episode_total_rewards[-1] + episode_reward / k
        else:
            average_episode_reward = episode_reward
        self.average_episode_total_rewards.append(average_episode_reward)

    def print_rewards(self, episode, print_epsilon=True, print_q_table=True):
        # print("Episode ", episode + 1)
        print("Total (discounted) reward of this episode: ", self.episode_total_rewards[episode])
        print("Average total reward over all episodes until now: ", self.average_episode_total_rewards[-1])

        print("Epsilon:", self.epsilon) if print_epsilon else None
        print("Q-table: ", self.qtable) if print_q_table else None

In [20]:
payoffs = np.array([[0, b[1]],[-c, b[1]-c]])
print(payoffs)

run = 0
np.random.seed(run)
random.seed(run)

agents = [Qlearner(seeded=True) for _ in range(4)] + [Qlearner(seeded=False) for _ in range(6)]
env = MatrixGame(payoffs, agents)

for episode in range(num_episodes):
    obs = env.reset()
    for round in range(num_rounds):
        action_rules = {agent: agent.select_action(obs[agent]) for agent in agents}
        next_obs, rewards = env.step(action_rules)

        for agent in agents:
            if round == num_rounds-1:
                agent.update(obs[agent], action_rules[agent], next_obs[agent], rewards[agent], done=True)
            else:
                agent.update(obs[agent], action_rules[agent], next_obs[agent], rewards[agent], done=False)
        obs = next_obs

average_agent_round_payoff = np.zeros(num_episodes//2)
for agent in agents:
    average_round_payoff = np.array(agent.episode_total_rewards[num_episodes//2:])/num_rounds
    print(average_round_payoff)
    average_agent_round_payoff += average_round_payoff


average_agent_round_payoff = (average_agent_round_payoff.sum()/(10*(num_episodes//2)))/(b[2]-c)

print(average_agent_round_payoff)

[[ 0  5]
 [-1  4]]
[0.175 0.175 0.295 0.425 0.05  0.125 0.145 0.175 0.225 0.195 0.145 0.025
 0.22  0.25  0.175 0.2   0.275 0.295 0.12  0.2   0.15  0.15  0.25  0.2
 0.2   0.195 0.22  0.175 0.15  0.125 0.12  0.175 0.17  0.12  0.2   0.1
 0.15  0.12  0.22  0.245 0.175 0.175 0.175 0.125 0.2   0.1   0.1   0.275
 0.275 0.07  0.17  0.2   0.275 0.225 0.075 0.15  0.15  0.145 0.125 0.1
 0.125 0.25  0.05  0.245 0.25  0.2   0.075 0.05  0.2   0.225 0.2   0.17
 0.1   0.15  0.125 0.2   0.075 0.15  0.195 0.25  0.15  0.275 0.175 0.125
 0.3   0.15  0.145 0.225 0.07  0.075 0.175 0.225 0.075 0.32  0.225 0.095
 0.2   0.125 0.1   0.2   0.17  0.25  0.125 0.17  0.1   0.225 0.15  0.125
 0.195 0.15  0.145 0.2   0.145 0.345 0.15  0.175 0.2   0.07  0.195 0.3
 0.1   0.2   0.275 0.1   0.17  0.195 0.175 0.125 0.2   0.175 0.225 0.07
 0.1   0.275 0.1   0.22  0.17  0.15  0.15  0.25  0.2   0.32  0.25  0.3
 0.095 0.17  0.125 0.375 0.15  0.12  0.05  0.1   0.25  0.12  0.15  0.245
 0.295 0.225 0.32  0.2   0.15  0.175 0.275 0